# Imports

In [2]:
import json
import os
import bz2
import io
from bz2 import BZ2File
import pandas as pd
import numpy as np

# Read dataset

In [3]:
input_dir = "data" # create folder 'data' containing .jsonl.bz2 file and update with your path 

In [4]:
# a helper function to get the lines from am archive
def read_jsonlines(bz2_file):
    text = bz2_file.read().decode('utf-8')
    for line in text.split('\n'):
        if line != '':
            yield line

In [1]:
archives = os.listdir(input_dir)

NameError: name 'os' is not defined

# Extract articles

In [6]:
%time
for archive in archives:
    
    # take only the transformed archives
    # open the archive
    f = BZ2File(os.path.join(input_dir, archive), 'r')
    articles = list(read_jsonlines(f))
    # get the list of articles it contains (= a json object on each line)
    print(archive, ':', len(articles), 'articles à extraire')
        
    # load the first 100 articles as json and access their attributes
    totalErrors = 0
    for a in articles:
            # decode the json string into an object (dict)
            json_article = json.loads(a)
            journal_.append(str(json_article["id"])[:3])
            date_.append(str(json_article["id"])[4:14])
            page_.append(str(json_article["pp"])[1:-1])
            try:
              text_.append(str(json_article["ft"]))
            except:
              text_.append("NULL")
              totalErrors = totalErrors + 1

    print("total errors", totalErrors)


CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 7.87 µs
GDL-1969.jsonl.bz2 : 50467 articles à extraire
GDL-1969-reduced.jsonl.bz2 : 50467 articles à extraire


# Create a pandas DataFrame

In [ ]:
dec = json.loads(articles[0])
dec["ft"]


In [7]:
df = pd.DataFrame.from_dict(
    {
        'journal': journal_,
        'date': date_,
        'page': page_,
        'text': text_
    })

In [8]:
df.head()

,journal,date,page,text
0,GDL,1969-07-07,1,Privée de gouvernement l'Italie est inquiète A...
1,GDL,1969-07-07,2,*• : & m wim W ^ i''***» A Cham on s'attaque a...
2,GDL,1969-07-07,3,Cent évêques européens aujourd'hui à Coire Com...
3,GDL,1969-07-07,4,Meta Antenen : record du monde AVEC 5046 POINT...
4,GDL,1969-07-07,5,GAÉTAN GANY A ESSAYÉ POUR VOUS Une Opel dans l...


In [9]:
len(df)

48111

# Clean and enrich the DataFrame 

In [10]:
def measure_articles(df):
    
    lengths = []
    
    for ind, row in df.iterrows():
        lengths.append(len(row['text']))
        
    return lengths

df['length'] = measure_articles(df)

In [11]:
import re
def handle_multiple_pages(df):
    
    page, ppage = [], []
    
    for ind, row in df.iterrows():
        
        found = re.findall('([0-9]+)', row['page'])
        if len(found) > 1:
            page.append(found[0])
            ppage.append(found[1])
        else:
            page.append(row['page'])
            ppage.append(np.nan)
        
    return page, ppage
    
page, ppage = handle_multiple_pages(df)
df['page'] = page
df['ppage'] = ppage

In [12]:
df.head()

,journal,date,page,text,length,ppage
0,GDL,1969-07-07,1,Privée de gouvernement l'Italie est inquiète A...,3667,NaN
1,GDL,1969-07-07,2,*• : & m wim W ^ i''***» A Cham on s'attaque a...,875,NaN
2,GDL,1969-07-07,3,Cent évêques européens aujourd'hui à Coire Com...,1063,NaN
3,GDL,1969-07-07,4,Meta Antenen : record du monde AVEC 5046 POINT...,3470,NaN
4,GDL,1969-07-07,5,GAÉTAN GANY A ESSAYÉ POUR VOUS Une Opel dans l...,9429,NaN


In [13]:
# Jeter les articles vides ou ne contenant que quelques caractères (p.ex titre des rubriques)
df = df[df['length'] > 50]

In [14]:
# Sauvegarder l'index
df['id'] = df.index

In [15]:
# Formater les types
df['date'] = pd.to_datetime(df['date'])
df['journal'] = df['journal'].astype('category')
#df['page'] = df['page'].astype('int')
#df['ppage'] = df['ppage'].astype('float')
df['text'] = df['text'].astype('str')

In [16]:
df = df[['id', 'journal', 'date', 'text', 'length']]

In [17]:
df.to_json('cleaned.json.bz2', compression = 'bz2')

In [18]:
df_lengths = df['length'].value_counts()

In [19]:
# Creating txt file
txt = open('data.txt', 'w+')
for ind, row in df.iterrows():
    if "nucleaire" in row['text'] or "nucléaire" in row['text']:
        date = str(row['date'])[0:10]
        txt.write('**** *'), txt.write(str(ind))
        txt.write(' *'), txt.write(str(row['page']))
        txt.write(' *'), txt.write(row['journal'])
        txt.write(' *'), txt.write(date), txt.write('\n')
        txt.write(row['text']), txt.write('\n')

txt.close()